# Import Thư viện

In [2]:
import pandas as pd
import re
from nltk import ngrams
from pyvi.pyvi import ViTokenizer


# Load dữ liệu train và stopwords

In [5]:
df = pd.read_csv('dataNLP.csv', header=None,names=['index','title','description'])
f = open('stopwords.txt','r')
stopwords = ['tập)', '(phần','dù',"div",'một','hai','ba','bốn','năm','sáu','bảy','tám','chín','mười']
for word in f:
    stopwords.append(word.strip()+ ' ')
f.close()
df.head(20)

,index,title,description
0,100000008,cô dâu 15 tuổi (my little bride),"sang min là dân mỹ thuật, tính đào hoa thích t..."
1,100000019,thất đại tội (24 tập) (the seven deadly sins),"24 tập phim theo chân elizabeth, công chúa thứ..."
2,100000026,cô gái ngoài hành tinh (13 tập) (brynhildr in ...,murakami luôn bị ám ảnh vì đã để người bạn thu...
3,100000031,10 phần yêu (love is not all around),"dù là cô nàng ""sát giai"", có nhiều mánh lới ""d..."
4,100000042,lớp học ám sát (phần 2) (assassination classro...,các học sinh lớp 3e-1 bước vào học kỳ thứ hai....
5,100000050,những giây cuối cùng (the final countdown),"năm 1980, trong lúc tập trận ở hawaii, một cơn..."
6,100000054,vua bịp đại chiến las vegas (the conmen in vegas),tài tử điện ảnh số một hồng kông tiếp tục có m...
7,100000059,vì yêu (a view of love),marc sở hữu gần như mọi thứ mà một người đàn ô...
8,100000061,cớm lưu manh (untouchable lawmen),"lee jung jin, một cảnh sát nắm đấm đi trước lờ..."
9,100000069,yêu không dễ (64 tập) (yeu khong de),"quy tụ dàn diễn viên “hot”, phim xoay quanh nh..."


# Tiền xử lý dữ liệu

In [55]:
def transform_row(row,_stopwords=True):
    # Xóa số dòng ở đầu câu
    row = row.split("(")[0]
    row = re.sub(r"[0-9]", "", row)
    if _stopwords :
        for word in stopwords:
            row = row.replace(word,"")
    # Xóa dấu chấm, phẩy, hỏi ở cuối câu
    row = re.sub(r"[\.,\?]+$", "", row)
    # Xóa các kí tự khác
    char = ''',.;":'?!()-<>/'''
    for c in char:
        row = row.replace(c," ")
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
#         .replace("!", " ").replace("?", " ") \
#         .replace("(", " ").replace(")", " ") \
#         .replace("-"," ").replace("div"," ") \
#         .replace("<"," ").replace(">"," ") \

    row = row.strip()
    return row 
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
    s = ''
    for c in input_str.decode('utf-8'):
        if c in s1:
            s += s0[s1.index(c)]
        else:
            s += c
    return s

df['title'] = df.title.apply(transform_row)
df['description'] = df.description.apply(transform_row)
df.head(20)

,index,title,description,title2,description2
0,100000008,cô dâu tuổi,sang min dân mỹ thuật tính đào hoa thích đuổi...,co dau tuoi,sang min dan my thuat tinh dao hoa thich duoi...
1,100000019,thất đại tội,tập phim chân elizabeth công chúa thứ vương ...,that dai toi,tap phim chan elizabeth cong chua thu vuong ...
2,100000026,cô gái ngoài hành tinh,murakami luôn ám ảnh người bạn thuở nhỏ kuro...,co gai ngoai hanh tinh,murakami luon am anh nguoi ban thuo nho kuro...
3,100000031,phần yêu,cô nàng sát giai mánh lới dắt mũi đàn ông...,phan yeu,co nang sat giai manh loi dat mui dan ong...
4,100000042,lớp học ám sát,học sinh lớp e bước học kỳ thứ họ nhận nhiệ...,lop hoc am sat,hoc sinh lop e buoc hoc ky thu ho nhan nhie...
5,100000050,giây cuối,trong tập trận ở hawaii cơn bão siêu nhiên x...,giay cuoi,trong tap tran o hawaii con bao sieu nhien x...
6,100000054,vua bịp đại chiến las vegas,tài tử điện ảnh số hồng kông tiếp tục màn hóa...,vua bip dai chien las vegas,tai tu dien anh so hong kong tiep tuc man hoa...
7,100000059,yêu,marc sở hữu gần mọi thứ người đàn ông trung n...,yeu,marc so huu gan moi thu nguoi dan ong trung n...
8,100000061,cớm lưu manh,lee jung jin cảnh sát nắm đấm đi lời nói jo...,com luu manh,lee jung jin canh sat nam dam di loi noi jo...
9,100000069,yêu dễ,quy tụ dàn diễn viên hot phim xoay quanh ân...,yeu de,quy tu dan dien vien hot phim xoay quanh an...


# Tách từ Tiếng Việt

In [4]:


def str_tokenizer(string):
    gram_str = list(ViTokenizer.tokenize(string.decode('utf-8')).split())
    return gram_str
df["token_title"] = df.title.apply(lambda t : str_tokenizer(t))
df["token_description"] = df.description.apply(lambda t : str_tokenizer(t))
df.head(20)

,index,title,description,token_title,token_description
0,100000008,cô dâu tuổi,sang min dân mỹ thuật tính đào hoa thích đuổi...,"[cô_dâu, tuổi]","[sang, min, dân, mỹ_thuật, tính, đào_hoa, thíc..."
1,100000019,thất đại tội,tập phim chân elizabeth công chúa thứ vương ...,"[thất, đại_tội]","[tập, phim, chân, elizabeth, công_chúa, thứ, v..."
2,100000026,cô gái ngoài hành tinh,murakami luôn ám ảnh người bạn thuở nhỏ kuro...,"[cô, gái, ngoài, hành_tinh]","[murakami, luôn, ám_ảnh, người, bạn, thuở, nhỏ..."
3,100000031,phần yêu,cô nàng sát giai mánh lới dắt mũi đàn ông...,"[phần, yêu]","[cô, nàng, sát, giai, mánh_lới, dắt_mũi, đàn_ô..."
4,100000042,lớp học ám sát,học sinh lớp e bước học kỳ thứ họ nhận nhiệ...,"[lớp_học, ám_sát]","[học_sinh, lớp, e, bước, học_kỳ, thứ, họ, nhận..."
5,100000050,giây cuối,trong tập trận ở hawaii cơn bão siêu nhiên x...,"[giây, cuối]","[trong, tập_trận, ở, hawaii, cơn, bão, siêu_nh..."
6,100000054,vua bịp đại chiến las vegas,tài tử điện ảnh số hồng kông tiếp tục màn hóa...,"[vua, bịp, đại_chiến, las_vegas]","[tài_tử, điện_ảnh, số, hồng, kông, tiếp_tục, m..."
7,100000059,yêu,marc sở hữu gần mọi thứ người đàn ông trung n...,[yêu],"[marc, sở_hữu, gần, mọi, thứ, người, đàn_ông, ..."
8,100000061,cớm lưu manh,lee jung jin cảnh sát nắm đấm đi lời nói jo...,"[cớm, lưu_manh]","[lee, jung, jin, cảnh_sát, nắm_đấm, đi, lời_nó..."
9,100000069,yêu dễ,quy tụ dàn diễn viên hot phim xoay quanh ân...,"[yêu, dễ]","[quy_tụ, dàn, diễn_viên, hot_phim, xoay, quanh..."


# Train data

In [5]:
from gensim.models import Word2Vec
df["context"] = df["token_title"] + df["token_description"]
train_data = df.context.tolist()
model = Word2Vec(train_data, size=100, window=10, min_count=3, workers=4, sg=1)


In [6]:
def similar_by_word(word):
    u = [x for x in model.wv.similar_by_word(word)]
    print word
    for i in u:
        print i[0],i[1]
strs = "lớp học ám sát (phần 4)"
s = transform_row(strs).decode('utf-8')
list = ViTokenizer.tokenize(s).split()
for word in list:
    similar_by_word(word)

lớp_học
giảng_dạy 0.976852059364
saburo 0.975131988525
tôn_thờ 0.974249303341
trí_tuệ 0.973558366299
makoto 0.973522245884
ten 0.973095178604
chế_giễu 0.972650527954
yato 0.972461342812
nam_sinh 0.972244083881
hướng_dẫn 0.972207784653
ám_sát
cia 0.94836461544
rửa_tiền 0.942266702652
an_ninh 0.936377644539
tối_mật 0.929275870323
imf 0.92894256115
truy_lùng 0.92882925272
đạn 0.924883782864
chiến_dịch 0.92385840416
sĩ_quan 0.920827746391
mua_bán 0.919986307621


# Tf-idf (Bonus)

In [7]:
#count
from collections import Counter
all_word = []
for sentence in train_data:
    all_word+=sentence
counts = Counter(all_word)
ndoc = 4719 
sentence_1 = df.token_description.iloc[1]
v =  set(sentence_1)
for word in v:
    print word, (ndoc / float(counts[word]))* (sentence_1.count(word) / float(len(sentence_1)))

người 0.0371530921545
bé_nhỏ 11.2357142857
hành_trình 0.501221455125
nào 0.596586599241
cân_bằng 33.7071428571
đi 0.15937183384
thất 7.93109243697
đại 1.21467181467
thế 0.564136282128
trong 0.043297550234
tội 3.21020408163
ngày 0.230476190476
thế_lực 1.26008010681
nhờ 0.97701863354
giành 1.03714285714
liones 67.4142857143
elizabeth 10.3714285714
về 0.0656419529837
công_chúa 3.09950738916
chân 0.97701863354
tay 0.406110154905
bạn 0.137861524978
nàng 0.373486347448
đặc_biệt 0.578663396689
vương_quốc 6.57700348432
hắc_ám 8.98857142857
thế_giới 0.262823725982
tập 2.93105590062
phim 0.0483256528418
giúp 0.615655577299
thứ 0.41106271777
còn 0.179532052501
tìm 0.154975369458


In [8]:
#Word2Vec cho tiếng việt không dấu

In [13]:
df2 = pd.read_csv('dataNLP.csv', header=None,names=['index','title','description'])

df2['title'] = df2.title.apply(transform_row)
df2['description'] = df2.description.apply(transform_row)
df2['title2'] = df2.title.apply(remove_accents)
df2['description2'] = df2.description.apply(remove_accents)
df2

,index,title,description,title2,description2
0,100000008,cô dâu tuổi,sang min là dân mỹ thuật tính đào hoa thích t...,co dau tuoi,sang min la dan my thuat tinh dao hoa thich t...
1,100000019,thất đại tội,tập phim theo chân elizabeth công chúa thứ ba...,that dai toi,tap phim theo chan elizabeth cong chua thu ba...
2,100000026,cô gái ngoài hành tinh,murakami luôn bị ám ảnh vì đã để người bạn thu...,co gai ngoai hanh tinh,murakami luon bi am anh vi da de nguoi ban thu...
3,100000031,phần yêu,dù là cô nàng sát giai có nhiều mánh lới d...,phan yeu,du la co nang sat giai co nhieu manh loi d...
4,100000042,lớp học ám sát,các học sinh lớp e bước vào học kỳ thứ hai h...,lop hoc am sat,cac hoc sinh lop e buoc vao hoc ky thu hai h...
5,100000050,những giây cuối cùng,năm trong lúc tập trận ở hawaii một cơn bão...,nhung giay cuoi cung,nam trong luc tap tran o hawaii mot con bao...
6,100000054,vua bịp đại chiến las vegas,tài tử điện ảnh số một hồng kông tiếp tục có m...,vua bip dai chien las vegas,tai tu dien anh so mot hong kong tiep tuc co m...
7,100000059,vì yêu,marc sở hữu gần như mọi thứ mà một người đàn ô...,vi yeu,marc so huu gan nhu moi thu ma mot nguoi dan o...
8,100000061,cớm lưu manh,lee jung jin một cảnh sát nắm đấm đi trước lờ...,com luu manh,lee jung jin mot canh sat nam dam di truoc lo...
9,100000069,yêu không dễ,quy tụ dàn diễn viên hot phim xoay quanh nh...,yeu khong de,quy tu dan dien vien hot phim xoay quanh nh...


In [37]:

from nltk import ngrams

def tokenize(string, n=1):
    gram_str = list(ngrams(string.split(), n))
    return [ " ".join(gram).lower() for gram in gram_str ]
tokenize('toi yeu viet nam',2)
df2['context']= df2.title.apply(lambda t:tokenize(t,1)) + df2.title.apply(lambda t:tokenize(t,2)) \
              + df2.description.apply(lambda t:tokenize(t,1))+ df2.description.apply(lambda t:tokenize(t,2)) \
              + df2.title2.apply(lambda t:tokenize(t,1)) + df2.title2.apply(lambda t:tokenize(t,2)) \
              + df2.description2.apply(lambda t:tokenize(t,1))+ df2.description2.apply(lambda t:tokenize(t,2))
 

In [40]:
from gensim.models import Word2Vec
train_data2 = df2.context.tolist()
model2 = Word2Vec(train_data2, size=100, window=10, min_count=3, workers=4, sg=1)


In [54]:
for x1,x2 in model2.wv.similar_by_word('chiến tranh'):
    print x1,x2 

chiến giữa 0.922339439392
đế chế 0.919656813145
tranh giành 0.919013082981
hòa bình 0.917619109154
thế kỷ 0.916240692139
giành quyền 0.914672195911
cuộc đấu 0.905402481556
chiến khốc 0.90378755331
khốc liệt 0.903047859669
chiến không 0.900208413601
